In [2]:
#from Tools.Tools.Htt_trigSF import Htt_trigSFRDFProducer
# import ROOT
# import JupyROOT.helpers.utils
# JupyROOT.helpers.utils.GetGeometryDrawer = lambda:None # magic fix to make ROOT faster
# import cppyy
import numpy as np
import hist
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
import math
import uproot

In [3]:
ar_raw = uproot.concatenate([f"root://eos.grif.fr//eos/grif/cms/llr/store/user/tcuisset/boostedTaus/2018_MC/GluGluToXToZZTo2B2Tau_M-1000/nano_{i}.root:Events" for i in [0]],
    filter_name=["event", "luminosityBlock",  "GenPart_pdgId", "GenPart_genPartIdxMother", "GenPart_eta", "GenPart_phi", "GenPart_*", "nGenVisTau", "GenVisTau*"], # "boostedTau*", "Muon_*", "Electron_*",
    how="zip")

In [34]:
statusFlags_doc = {0 : "isPrompt", 1 : "isDecayedLeptonHadron", 2 : "isTauDecayProduct", 3 : "isPromptTauDecayProduct", 4 : "isDirectTauDecayProduct", 5 : "isDirectPromptTauDecayProduct", 6 : "isDirectHadronDecayProduct", 7 : "isHardProcess", 8 : "fromHardProcess", 9 : "isHardProcessTauDecayProduct", 10 : "isDirectHardProcessTauDecayProduct", 11 : "fromHardProcessBeforeFSR", 12 : "isFirstCopy", 13 : "isLastCopy", 14 : "isLastCopyBeforeFSR"}
def readStatusFlagsBits(val):
    s = ""
    for bitn in statusFlags_doc.keys():
        is_bit_set = (val & (1 << bitn)) != 0
        if is_bit_set:
            s += statusFlags_doc[bitn] + ", "
    return s

In [4]:
evt = ar_raw[ar_raw.event == 1311][0]
evt

<Record {luminosityBlock: 2, event: 1311, ...} type='{luminosityBlock: uint...'>

In [5]:
evt.GenPart[["pdgId", "genPartIdxMother", "pt"]]

<Array [{pdgId: 21, ...}, {...}, ..., {...}] type='57 * {pdgId: int32, genP...'>

In [6]:
gen_tau = (abs(evt.GenPart.pdgId) == 15) & (evt.GenPart[evt.GenPart.genPartIdxMother].pdgId == 23) 
print(np.where(gen_tau))
evt.GenPart[gen_tau]

(<Array [13, 14] type='2 * int64'>,)


<Array [{genPartIdxMother: 11, ...}, ...] type='2 * {genPartIdxMother: int1...'>

In [32]:
readStatusFlagsBits(13884), readStatusFlagsBits(4481), 

('isTauDecayProduct, isPromptTauDecayProduct, isDirectTauDecayProduct, isDirectPromptTauDecayProduct, isHardProcessTauDecayProduct, isDirectHardProcessTauDecayProduct, isFirstCopy, isLastCopy, ',
 'isPrompt, isHardProcess, fromHardProcess, isFirstCopy, ')

In [40]:
idx=15
print(np.where(evt.GenPart.genPartIdxMother == idx))
evt.GenPart[evt.GenPart.genPartIdxMother == idx][["genPartIdxMother", "pdgId", "eta", "statusFlags"]]

(<Array [25, 26] type='2 * int64'>,)


<Array [{genPartIdxMother: 15, ...}, ...] type='2 * {genPartIdxMother: int1...'>

In [39]:
evt.GenVisTau

<Array [{status: 0, charge: -1, ...}] type='1 * {status: uint8, charge: int...'>

In [18]:
evt.GenPart[29]

<Record {genPartIdxMother: 14, ...} type='{genPartIdxMother: int16, statusF...'>

In [20]:
evt.GenPart[15]

<Record {genPartIdxMother: 11, ...} type='{genPartIdxMother: int16, statusF...'>

In [30]:
print(np.where(evt.GenPart.genPartIdxMother == 19))
evt.GenPart[evt.GenPart.genPartIdxMother == 19]

(<Array [55, 56, 57] type='3 * int64'>,)


<Array [{genPartIdxMother: 19, ...}, ...] type='3 * {genPartIdxMother: int1...'>

In [29]:
evt.GenVisTau

<Array [] type='0 * {status: uint8, charge: int16, genPartIdxMother: int16,...'>

In [31]:
ev2 = ar_raw[-1]
ev2

<Record {luminosityBlock: 28, event: 27997, ...} type='{luminosityBlock: ui...'>

In [33]:
gen_tau = (abs(evt.GenPart.pdgId) == 15) & (evt.GenPart[evt.GenPart.genPartIdxMother].pdgId == 23) 
print(np.where(gen_tau))
evt.GenPart[gen_tau]

(<Array [13, 14] type='2 * int64'>,)


<Array [{genPartIdxMother: 11, ...}, ...] type='2 * {genPartIdxMother: int1...'>

In [34]:
print(np.where(evt.GenPart.genPartIdxMother == 13))
evt.GenPart[evt.GenPart.genPartIdxMother == 13]

(<Array [15, 19] type='2 * int64'>,)


<Array [{genPartIdxMother: 13, ...}, ...] type='2 * {genPartIdxMother: int1...'>

In [42]:
import awkward as ak
list(np.unique(ak.flatten(ar_raw.GenPart[(ar_raw.GenPart[ar_raw.GenPart.genPartIdxMother].pdgId == 15)].pdgId)))

[-2112,
 -323,
 -321,
 -313,
 -311,
 -213,
 -211,
 -15,
 -14,
 -13,
 -12,
 -11,
 -2,
 -1,
 1,
 2,
 11,
 13,
 15,
 16,
 21,
 22,
 111,
 130,
 211,
 213,
 221,
 223,
 310,
 311,
 321,
 331,
 1114]

In [49]:
weird = (ar_raw.GenPart[ar_raw.GenPart.genPartIdxMother].pdgId == 15) & (ar_raw.GenPart.pdgId == 1)

In [55]:
evt = ar_raw[ak.any(weird, axis=-1)][0]

In [56]:
evt.GenPart[(evt.GenPart[evt.GenPart.genPartIdxMother].pdgId == 15) & (evt.GenPart.pdgId == 1)]

<Array [{genPartIdxMother: 16, ...}] type='1 * {genPartIdxMother: int16, st...'>

In [65]:
print(np.where(evt.GenPart.genPartIdxMother == 16))
evt.GenPart[evt.GenPart.genPartIdxMother == 16][["pdgId", "pt"]]

(<Array [18, 21, 22, 23] type='4 * int64'>,)


<Array [{pdgId: 15, pt: 824}, ..., {...}] type='4 * {pdgId: int32, pt: floa...'>

In [67]:
evt.GenPart[16]

<Record {genPartIdxMother: 12, ...} type='{genPartIdxMother: int16, statusF...'>

In [63]:
readStatusFlagsBits(8193)

'isPrompt, isLastCopy, '